## Assignment 3 ##

Your Name:

## Assignment Question ##

Using the below data set, perform LDA topic modeling to identify multiple latent topics inside.

The number of topipcs is not fixed, it is up to you to decide how many topics to go with.

## Grading Guidelines: ##

You need to show all the steps (Codes & outputs) from uploading the data set to performing topic modeling to derive topics with keywords.

DO NOT CLEAR THE OUTPUTS (Leave the outputs printed).


## Step 1: Load the dataset

The dataset we'll use is a list of news headlines published over a period of 15 years. 

We'll start by loading it from the `abcnews-date-text.csv` file.

In [1]:
#connect Colab to your Google Drive.
from google.colab import drive
import os
drive.mount('/content/gdrive')

#Load the dataset from the CSV and save it to 'data_text'

import pandas as pd
news=pd.read_csv('/content/gdrive/My Drive/CIS NLP Data Sets/abcnews-date-text.csv')
print (news.iloc[:10,:])
#How many columns and rows?
print ("Shape:", news.shape)
#Column names?
print ("Column Names",news.columns.values)

Mounted at /content/gdrive
   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers
5      20030219                  ambitious olsson wins triple jump
6      20030219         antic delighted with record breaking barca
7      20030219  aussie qualifier stosur wastes four memphis match
8      20030219       aust addresses un security council over iraq
9      20030219         australia is locked into war timetable opp
Shape: (1103665, 2)
Column Names ['publish_date' 'headline_text']


## Step 2: Data Preprocessing ##

We will perform the following steps:

The order of the pre-processing steps doesn't have to be in this way.

It is up to you whether you start tokenizing first or other processing steps or at the same time.

HOWEVER, make sure that all the below steps are performed and applied to the headline text.

* **Tokenization** 
* **Lowercasing** 
* **remove punctuations**
* **Words that have fewer than 3 characters are removed**
* **stopwords are removed**
* **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.

**Lemmatization code is give below, use the below code for lemmatization.**

In [7]:
from nltk.stem import WordNetLemmatizer
print(WordNetLemmatizer().lemmatize('did', pos = 'v'))

do


In [3]:
!pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [8]:
#lemmatization (simplified version)
def lemmatize_text(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

# and then remove stop words and words fewer than 2 characters.
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_text(token))
    return result

news['headline_text_processed'] = news['headline_text'].map(preprocess)

print (news.iloc[:10,:])
print ("Shape:", news.shape)
print ("Column Names",news.columns.values)


   publish_date                                      headline_text  \
0      20030219  aba decides against community broadcasting lic...   
1      20030219     act fire witnesses must be aware of defamation   
2      20030219     a g calls for infrastructure protection summit   
3      20030219           air nz staff in aust strike for pay rise   
4      20030219      air nz strike to affect australian travellers   
5      20030219                  ambitious olsson wins triple jump   
6      20030219         antic delighted with record breaking barca   
7      20030219  aussie qualifier stosur wastes four memphis match   
8      20030219       aust addresses un security council over iraq   
9      20030219         australia is locked into war timetable opp   

                             headline_text_processed  
0            [decide, community, broadcast, licence]  
1                       [witness, aware, defamation]  
2         [call, infrastructure, protection, summit]  
3        

## Step 3: Bag of words on the dataset

* 3-1. Dictionary

Create a dictionary from pre-processed headline texts containing the number of times a word appears in the training set. 

To do that, let's pass your pre-processed headline texts to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [ ]:
dictionary = gensim.corpora.Dictionary()

In [10]:
dictionary = gensim.corpora.Dictionary(news['headline_text_processed'])

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

print (len(dictionary))

0 broadcast
1 community
2 decide
3 licence
4 aware
5 defamation
6 witness
7 call
8 infrastructure
9 protection
10 summit
77773


* 3-2. Gensim filter_extremes

[`filter_extremes(no_below=i, no_above=j, keep_n=k) where i,j,k can be integers or fractions.`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [11]:
#Remove very rare and very common words using filter_extremes():
dictionary.filter_extremes(no_below=500, no_above=0.70)
print (len(dictionary))

1872


* 3-3. Gensim doc2bow

* Gensim doc2bow (pass the tokenized words to doc2bow and convert those to vectors.)

* Caution: No further preprocessing should be done such as tokenization, lemmatization, and etc before initiating this.

In [12]:
bow_corpus = [dictionary.doc2bow(doc) for doc in news['headline_text_processed']]
print (len(bow_corpus))
print (bow_corpus[0])

1103665
[(0, 1), (1, 1), (2, 1)]


## Step 4: Running LDA using Bag of Words ##

Perform LDA model on your final corpus.


In [13]:
#Run LDA model on the final corpus.

#num_topics: the number of latent topics to be extracted from the corpus.
#id2word: mapping from word ids (integers) to words (strings).
# Some other parameters. See the document explanations for more details.

lda_model_tfidf = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary
                                             )

In [14]:
import re
for idx, topic in lda_model_tfidf.print_topics(-1, num_words=10):
    #print out topic numbers and keywords.
#    print('Topic: {} Word: {}'.format(idx, topic))
    
    #print out keywords only (without probability)
    key_words_only = " ".join(re.findall("[a-zA-Z]+", topic))
    print ('Topic:',idx,key_words_only)

Topic: 0 melbourne attack kill change labor arrest dead report shoot victorian
Topic: 1 live tasmanian show interview bank energy make turn rule reveal
Topic: 2 australia trump world home open south time family league final
Topic: 3 police charge court murder australian death woman jail turnbull accuse
Topic: 4 test canberra break lose korea face john talk lead violence
Topic: 5 sydney school perth call miss deal national flood work children
Topic: 6 say government donald crash die market rise high record price
Topic: 7 north adelaide year indigenous win tasmania west concern country vote
Topic: 8 election house coast brisbane years power china leave life fight
Topic: 9 queensland help hospital fund council health return abuse victoria want


### Step 5: label the topics ###

Using the keywords in each topic , what topics were you able to infer?
You should write down the inferred topic labels below.

* 0: 
* 1: 
* 2: 
* ...